In [1]:
#imports
import lightgbm as lgb
import numpy as np
import pandas as pd
import sqlalchemy
from catboost import CatBoostRegressor
from hyperopt import hp
from hyperopt.pyll import scope
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from catboost import CatBoostRegressor

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from models.hyperopt_model import HyperoptModel
from models.utils import CustomDataFrameMapper
from models.utils import _tree_features_transformations

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# read the data
preprocessed_data = pd.read_pickle('./data/preprocesed_data.200.20.big.pd')

# Create a dataframe
df = pd.DataFrame(preprocessed_data)

In [3]:
# Create two new dataframes, one with the training rows, one with the test rows
train, test=train_test_split(df, test_size=0.2, random_state=0)

In [4]:
# Show the number of observations for the test and training dataframes
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 114411
Number of observations in the test data: 28603


In [5]:
    #some parameters
    DO_LOWESS = False
    hyperopt = True
    # Run CatBoostRegressor with hyperopt optimization
    answer_features_transformations, \
    question_features_transformations, \
    time_features_transformations, \
    user_features_transformations = _tree_features_transformations()
    
    cat = HyperoptModel(train.copy(), test.copy(), 'cat', cv=3)
    cat.raw_features = []
    cat.pipeline = Pipeline([
        ('prepare_features', FeatureUnion([
            ('user_features', CustomDataFrameMapper(user_features_transformations)),
            ('time_features', CustomDataFrameMapper(time_features_transformations)),
            ('answer_features', CustomDataFrameMapper(answer_features_transformations)),
            ('question_features', CustomDataFrameMapper(question_features_transformations))
        ])),
        ('estimate', CatBoostRegressor(**{'bagging_temperature': 0.29793733267072053,
                                          'iterations': 5000,
                                          'l2_leaf_reg': 1.5511980979084095,
                                          'learning_rate': 0.08683045812519587,
                                          'loss_function': 'RMSE',
                                          'random_seed': 0,
                                          'random_strength': 20,
                                          'verbose': True,
                                          'train_dir': 'outputs/cat'}))
    ])
    
    for transformer in cat.pipeline.named_steps['prepare_features'].transformer_list:
        cat.raw_features += [t[0] if isinstance(t[0], str) else t[0][0] for t in transformer[1].features]
    
    cat.space = {
        'estimate__iterations': hp.choice('estimate__iterations', [5000]),
        'estimate__loss_function': hp.choice('estimate__loss_function', ['RMSE']),
        'estimate__train_dir': hp.choice('estimate__train_dir', ['outputs/cat']),
        'estimate__thread_count': hp.choice('estimate__thread_count', [4]),
        'estimate__used_ram_limit': hp.choice('estimate__used_ram_limit', [1024 * 1024 * 1024 * 4]),  # 4gb
        'estimate__random_seed': hp.choice('estimate__random_seed', [0]),

        'estimate__learning_rate': hp.loguniform('estimate__learning_rate', -5, 0),
        'estimate__random_strength': hp.choice('estimate__random_strength', [1, 20]),
        'estimate__l2_leaf_reg': hp.loguniform('estimate__l2_leaf_reg', 0, np.log(10)),
        'estimate__bagging_temperature': hp.uniform('estimate__bagging_temperature', 0, 1),
    }

    """
    find the best number of trees (following https://tech.yandex.com/catboost/doc/dg/concepts/parameter-tuning_trees-number-docpage/#parameter-tuning_trees-number)
    need to also substitute the CatBoostRegressor parameters with:
        ('estimate', CatBoostRegressor(iterations=10000, loss_function='RMSE', auto_stop_pval=1e-4, use_best_model=True, train_dir='outputs/cat_trees', verbose=True))
    """
    # num_trees_train, num_trees_eval = train_test_split(train, test_size=0.2, random_state=0)
    # X = num_trees_train[list(filter(lambda column: column in cat.raw_features, cat.train.columns))]
    # y = num_trees_train['score']
    #
    # eval_X = num_trees_eval[list(filter(lambda column: column in cat.raw_features, cat.train.columns))]
    # eval_X = cat.pipeline.named_steps['prepare_features'].fit_transform(eval_X)
    # eval_set = (eval_X, num_trees_eval['score'])
    #
    # cat.pipeline.fit(X=X, y=y, estimate__eval_set=eval_set)
    # cat.model = cat.pipeline
    # print(cat.model.named_steps['estimate'].get_params())
    # cat.model.named_steps['estimate'].save_model('tmp/cat.model'.encode('utf-8'))

    if hyperopt:
        cat.run(do_lowess=DO_LOWESS)
    else:
        # train with default params
        cat.pipeline.fit(X=cat.X_train, y=cat.y_train)
        cat.model = cat.pipeline
        cat.stats()

        # joblib.dump(cat, './outputs/models/%s.pckl' % cat.output_prefix)
        # cat = joblib.load('./outputs/models/%s.pckl' % cat.output_prefix)

        cat.plot_predicted_vs_actual(do_lowess=False)
        cat.plot_residuals(r_type='raw', do_lowess=False)
        cat.plot_feature_importance()

Performing parameters optimization...
Borders generated
0:	learn 0.5178346999passed: 0.202 sec	total: 1.01s	remaining: 1h 24m 20s
1:	learn 0.4884888767passed: 0.17 sec	total: 1.18s	remaining: 49m 16s
2:	learn 0.4609421967passed: 0.169 sec	total: 1.35s	remaining: 37m 33s
3:	learn 0.4349519119passed: 0.153 sec	total: 1.5s	remaining: 31m 20s
4:	learn 0.4106534423passed: 0.15 sec	total: 1.66s	remaining: 27m 34s
5:	learn 0.3879415841passed: 0.156 sec	total: 1.81s	remaining: 25m 8s
6:	learn 0.366667651passed: 0.164 sec	total: 1.98s	remaining: 23m 30s
7:	learn 0.3467003202passed: 0.159 sec	total: 2.14s	remaining: 22m 13s
8:	learn 0.3280252529passed: 0.167 sec	total: 2.3s	remaining: 21m 17s
9:	learn 0.3104691202passed: 0.145 sec	total: 2.45s	remaining: 20m 22s
10:	learn 0.294156068passed: 0.16 sec	total: 2.61s	remaining: 19m 43s
11:	learn 0.2789259477passed: 0.152 sec	total: 2.76s	remaining: 19m 8s
12:	learn 0.2645023514passed: 0.16 sec	total: 2.92s	remaining: 18m 41s
13:	learn 0.2511861545pas

114:	learn 0.09064192634passed: 0.16 sec	total: 19.7s	remaining: 13m 56s
115:	learn 0.09061635156passed: 0.167 sec	total: 19.9s	remaining: 13m 55s
116:	learn 0.09059286474passed: 0.158 sec	total: 20s	remaining: 13m 55s
117:	learn 0.09057308917passed: 0.169 sec	total: 20.2s	remaining: 13m 54s
118:	learn 0.09056480824passed: 0.17 sec	total: 20.3s	remaining: 13m 54s
119:	learn 0.09050618196passed: 0.174 sec	total: 20.5s	remaining: 13m 54s
120:	learn 0.09044762789passed: 0.169 sec	total: 20.7s	remaining: 13m 54s
121:	learn 0.09041774081passed: 0.163 sec	total: 20.9s	remaining: 13m 53s
122:	learn 0.09033510392passed: 0.173 sec	total: 21s	remaining: 13m 53s
123:	learn 0.0902884934passed: 0.154 sec	total: 21.2s	remaining: 13m 52s
124:	learn 0.09027686923passed: 0.173 sec	total: 21.4s	remaining: 13m 52s
125:	learn 0.09023567546passed: 0.166 sec	total: 21.5s	remaining: 13m 52s
126:	learn 0.09020481577passed: 0.181 sec	total: 21.7s	remaining: 13m 52s
127:	learn 0.09019778071passed: 0.167 sec	tot

226:	learn 0.08783430945passed: 0.174 sec	total: 38.2s	remaining: 13m 22s
227:	learn 0.08780551237passed: 0.181 sec	total: 38.4s	remaining: 13m 23s
228:	learn 0.08776654333passed: 0.177 sec	total: 38.5s	remaining: 13m 23s
229:	learn 0.08772618716passed: 0.182 sec	total: 38.7s	remaining: 13m 23s
230:	learn 0.08769846931passed: 0.166 sec	total: 38.9s	remaining: 13m 23s
231:	learn 0.08766601428passed: 0.16 sec	total: 39.1s	remaining: 13m 22s
232:	learn 0.08763029259passed: 0.166 sec	total: 39.2s	remaining: 13m 22s
233:	learn 0.08760220098passed: 0.16 sec	total: 39.4s	remaining: 13m 22s
234:	learn 0.08756716893passed: 0.15 sec	total: 39.5s	remaining: 13m 21s
235:	learn 0.08752744397passed: 0.162 sec	total: 39.7s	remaining: 13m 21s
236:	learn 0.08749251891passed: 0.166 sec	total: 39.9s	remaining: 13m 21s
237:	learn 0.0874484663passed: 0.155 sec	total: 40s	remaining: 13m 20s
238:	learn 0.08742009117passed: 0.166 sec	total: 40.2s	remaining: 13m 20s
239:	learn 0.08739697784passed: 0.15 sec	tot

340:	learn 0.08494569589passed: 0.163 sec	total: 56.6s	remaining: 12m 53s
341:	learn 0.08492371944passed: 0.154 sec	total: 56.8s	remaining: 12m 53s
342:	learn 0.08490350272passed: 0.164 sec	total: 57s	remaining: 12m 53s
343:	learn 0.08489160275passed: 0.166 sec	total: 57.1s	remaining: 12m 53s
344:	learn 0.08487634793passed: 0.151 sec	total: 57.3s	remaining: 12m 52s
345:	learn 0.08485038231passed: 0.179 sec	total: 57.5s	remaining: 12m 52s
346:	learn 0.08483425929passed: 0.163 sec	total: 57.6s	remaining: 12m 52s
347:	learn 0.0848186131passed: 0.158 sec	total: 57.8s	remaining: 12m 52s
348:	learn 0.08480255145passed: 0.16 sec	total: 57.9s	remaining: 12m 52s
349:	learn 0.08479044987passed: 0.156 sec	total: 58.1s	remaining: 12m 51s
350:	learn 0.08477148207passed: 0.16 sec	total: 58.3s	remaining: 12m 51s
351:	learn 0.08475432715passed: 0.17 sec	total: 58.4s	remaining: 12m 51s
352:	learn 0.08473607042passed: 0.163 sec	total: 58.6s	remaining: 12m 51s
353:	learn 0.08472167579passed: 0.158 sec	to

452:	learn 0.08337380087passed: 0.166 sec	total: 1m 14s	remaining: 12m 31s
453:	learn 0.08336226453passed: 0.168 sec	total: 1m 15s	remaining: 12m 31s
454:	learn 0.08335267138passed: 0.148 sec	total: 1m 15s	remaining: 12m 31s
455:	learn 0.08333885822passed: 0.153 sec	total: 1m 15s	remaining: 12m 31s
456:	learn 0.08332462126passed: 0.16 sec	total: 1m 15s	remaining: 12m 30s
457:	learn 0.0833102794passed: 0.156 sec	total: 1m 15s	remaining: 12m 30s
458:	learn 0.08329488961passed: 0.164 sec	total: 1m 15s	remaining: 12m 30s
459:	learn 0.08328511728passed: 0.163 sec	total: 1m 16s	remaining: 12m 30s
460:	learn 0.08327509692passed: 0.163 sec	total: 1m 16s	remaining: 12m 30s
461:	learn 0.08326547263passed: 0.148 sec	total: 1m 16s	remaining: 12m 29s
462:	learn 0.08325682051passed: 0.182 sec	total: 1m 16s	remaining: 12m 29s
463:	learn 0.08324422285passed: 0.183 sec	total: 1m 16s	remaining: 12m 29s
464:	learn 0.0832327538passed: 0.17 sec	total: 1m 16s	remaining: 12m 29s
465:	learn 0.08322077855passe

...

4488:	learn 0.06966076031passed: 0.207 sec	total: 16m 17s	remaining: 1m 51s
4489:	learn 0.06965803938passed: 0.217 sec	total: 16m 17s	remaining: 1m 51s
4490:	learn 0.06965609326passed: 0.207 sec	total: 16m 17s	remaining: 1m 50s
4491:	learn 0.0696530136passed: 0.207 sec	total: 16m 17s	remaining: 1m 50s
4492:	learn 0.06965028565passed: 0.213 sec	total: 16m 18s	remaining: 1m 50s
4493:	learn 0.06964767077passed: 0.208 sec	total: 16m 18s	remaining: 1m 50s
4494:	learn 0.06964498019passed: 0.21 sec	total: 16m 18s	remaining: 1m 49s
4495:	learn 0.06964255392passed: 0.218 sec	total: 16m 18s	remaining: 1m 49s
4496:	learn 0.06964059267passed: 0.201 sec	total: 16m 18s	remaining: 1m 49s
4497:	learn 0.0696390248passed: 0.206 sec	total: 16m 19s	remaining: 1m 49s
4498:	learn 0.06963725204passed: 0.217 sec	total: 16m 19s	remaining: 1m 49s
4499:	learn 0.06963468866passed: 0.205 sec	total: 16m 19s	remaining: 1m 48s
4500:	learn 0.06963307919passed: 0.207 sec	total: 16m 19s	remaining: 1m 48s
4501:	learn 0.0

4596:	learn 0.06943666392passed: 0.208 sec	total: 16m 40s	remaining: 1m 27s
4597:	learn 0.06943460367passed: 0.211 sec	total: 16m 40s	remaining: 1m 27s
4598:	learn 0.0694322302passed: 0.21 sec	total: 16m 40s	remaining: 1m 27s
4599:	learn 0.06942912082passed: 0.213 sec	total: 16m 40s	remaining: 1m 27s
4600:	learn 0.06942734271passed: 0.21 sec	total: 16m 40s	remaining: 1m 26s
4601:	learn 0.06942487598passed: 0.207 sec	total: 16m 41s	remaining: 1m 26s
4602:	learn 0.06942255023passed: 0.205 sec	total: 16m 41s	remaining: 1m 26s
4603:	learn 0.06942013319passed: 0.211 sec	total: 16m 41s	remaining: 1m 26s
4604:	learn 0.06941862141passed: 0.206 sec	total: 16m 41s	remaining: 1m 25s
4605:	learn 0.06941638382passed: 0.212 sec	total: 16m 41s	remaining: 1m 25s
4606:	learn 0.06941486845passed: 0.203 sec	total: 16m 42s	remaining: 1m 25s
4607:	learn 0.0694127209passed: 0.212 sec	total: 16m 42s	remaining: 1m 25s
4608:	learn 0.06941150895passed: 0.204 sec	total: 16m 42s	remaining: 1m 25s
4609:	learn 0.06

4705:	learn 0.06921634794passed: 0.215 sec	total: 17m 3s	remaining: 1m 3s
4706:	learn 0.06921485341passed: 0.209 sec	total: 17m 3s	remaining: 1m 3s
4707:	learn 0.06921193831passed: 0.212 sec	total: 17m 3s	remaining: 1m 3s
4708:	learn 0.06920880735passed: 0.209 sec	total: 17m 3s	remaining: 1m 3s
4709:	learn 0.06920679407passed: 0.215 sec	total: 17m 3s	remaining: 1m 3s
4710:	learn 0.06920528157passed: 0.205 sec	total: 17m 4s	remaining: 1m 2s
4711:	learn 0.06920333489passed: 0.212 sec	total: 17m 4s	remaining: 1m 2s
4712:	learn 0.06920173931passed: 0.206 sec	total: 17m 4s	remaining: 1m 2s
4713:	learn 0.06919941008passed: 0.214 sec	total: 17m 4s	remaining: 1m 2s
4714:	learn 0.06919805479passed: 0.208 sec	total: 17m 5s	remaining: 1m 1s
4715:	learn 0.06919561622passed: 0.209 sec	total: 17m 5s	remaining: 1m 1s
4716:	learn 0.06919368114passed: 0.213 sec	total: 17m 5s	remaining: 1m 1s
4717:	learn 0.06919166136passed: 0.205 sec	total: 17m 5s	remaining: 1m 1s
4718:	learn 0.06918981809passed: 0.214

4816:	learn 0.06898457315passed: 0.206 sec	total: 17m 26s	remaining: 39.8s
4817:	learn 0.06898284816passed: 0.219 sec	total: 17m 26s	remaining: 39.5s
4818:	learn 0.06897942889passed: 0.213 sec	total: 17m 26s	remaining: 39.3s
4819:	learn 0.06897611047passed: 0.205 sec	total: 17m 27s	remaining: 39.1s
4820:	learn 0.06897415806passed: 0.208 sec	total: 17m 27s	remaining: 38.9s
4821:	learn 0.06897161041passed: 0.207 sec	total: 17m 27s	remaining: 38.7s
4822:	learn 0.06897074291passed: 0.207 sec	total: 17m 27s	remaining: 38.5s
4823:	learn 0.06896807507passed: 0.215 sec	total: 17m 28s	remaining: 38.2s
4824:	learn 0.06896609009passed: 0.204 sec	total: 17m 28s	remaining: 38s
4825:	learn 0.06896425612passed: 0.222 sec	total: 17m 28s	remaining: 37.8s
4826:	learn 0.06896279212passed: 0.2 sec	total: 17m 28s	remaining: 37.6s
4827:	learn 0.06895968851passed: 0.204 sec	total: 17m 28s	remaining: 37.4s
4828:	learn 0.06895569054passed: 0.21 sec	total: 17m 29s	remaining: 37.1s
4829:	learn 0.06895316069passe

4926:	learn 0.06874858378passed: 0.209 sec	total: 17m 49s	remaining: 15.8s
4927:	learn 0.0687466497passed: 0.217 sec	total: 17m 49s	remaining: 15.6s
4928:	learn 0.06874466116passed: 0.208 sec	total: 17m 50s	remaining: 15.4s
4929:	learn 0.06874321384passed: 0.205 sec	total: 17m 50s	remaining: 15.2s
4930:	learn 0.06874066987passed: 0.215 sec	total: 17m 50s	remaining: 15s
4931:	learn 0.0687385498passed: 0.207 sec	total: 17m 50s	remaining: 14.8s
4932:	learn 0.06873550673passed: 0.213 sec	total: 17m 50s	remaining: 14.5s
4933:	learn 0.06873275986passed: 0.223 sec	total: 17m 51s	remaining: 14.3s
4934:	learn 0.06873176747passed: 0.2 sec	total: 17m 51s	remaining: 14.1s
4935:	learn 0.06873033322passed: 0.21 sec	total: 17m 51s	remaining: 13.9s
4936:	learn 0.0687287034passed: 0.203 sec	total: 17m 51s	remaining: 13.7s
4937:	learn 0.06872673935passed: 0.215 sec	total: 17m 52s	remaining: 13.5s
4938:	learn 0.06872466273passed: 0.217 sec	total: 17m 52s	remaining: 13.2s
4939:	learn 0.06872241327passed: 